# Chapter 9: LangChain Framework Integrations


# Environment Setup

In [ ]:
import importlib
import subprocess
import sys

libraries = ["langchain", "langchain_community", "huggingface_hub", "langchain_openai"]

for library in libraries:
    try:
        # Try to import the library
        module = importlib.import_module(library)
        print(f"Library {library} version: {module.__version__}")
    except ImportError:
        # If library is not installed, attempt to install it
        print(f"Library {library} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        # After installing, import again and print the version
        module = importlib.import_module(library)
        # print(f"Library {library} version after installation: {module.__version__}")
    except AttributeError:
        # If library doesn't have __version__ attribute
        print(f"Library {library} does not have a __version__ attribute.")

Library langchain version: 0.3.23
Library langchain_community not found. Installing...
Library huggingface_hub version: 0.30.2
Library langchain_openai not found. Installing...


In [ ]:
import os
from langchain.llms import HuggingFaceHub
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from google.colab import userdata

**Connect to HuggingFace LLM**

In [ ]:
# Load LLaMA 4 model from Hugging Face Hub (make sure it's a chat-compatible LLaMA4 model)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
llm = HuggingFaceHub(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",  # Adjust if you're using LLaMA 4 when it's available
    model_kwargs={
        "temperature": 0.7,
        "max_new_tokens": 256,
        "top_p": 0.9,
        "repetition_penalty": 1.1
    }
)

<ipython-input-3-16d6ed11a576>:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


Azure OpenAI LLM

In [ ]:
import os
from langchain_openai import AzureOpenAI
from google.colab import userdata

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZ_OPENAI_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] =  "https://azopenai-demo.openai.azure.com/"

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
embedding_model = AzureOpenAI(deployment_name="dp-text-embedding-ada-002", model_name="text-embedding-ada-002")

## Working with External APIs

Go to https://home.openweathermap.org/api_keys and create a free account to generate a free key

In [ ]:
openWeatherAPI_key = "<openWeatherAPI key>"

In [ ]:
from langchain.tools import BaseTool
import requests

class WeatherTool(BaseTool):
    name: str = "weather_tool"
    description: str = "Fetches weather information for a given city."

    def _run(self, city: str) -> str:
        api_key = openWeatherAPI_key  # <-- make sure to define your API key properly
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
        response = requests.get(url)
        data = response.json()
        if response.status_code == 200:
            return f"The weather in {city} is {data['weather'][0]['description']} with a temperature of {data['main']['temp']} °C."
        else:
            return f"Failed to fetch weather data: {data.get('message', 'Unknown error')}"

    async def _arun(self, city: str) -> str:
        raise NotImplementedError("Async not implemented yet.")


In [ ]:
weather_tool = WeatherTool()
city = "London"
result = weather_tool._run(city)
print(result)

The weather in London is overcast clouds with a temperature of 7.75 °C.


## Use the Tool in an Agent

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.llms import OpenAI

weather_tool = WeatherTool()

agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

response = agent.invoke("What's the weather like in Dublin today?")
print(response)

<ipython-input-12-e9672950623b>:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
<ipython-input-12-e9672950623b>:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What's the weather like in Dublin today?")


I am unable to answer the original question as I am unable to fetch the weather for Dublin.


## Sentiment Analysis with a PyTorch Model


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from langchain.tools import BaseTool
from typing import Optional
from pydantic import Field, model_validator

class SentimentAnalysisTool(BaseTool):
    name: str = "sentiment_analysis"
    description: str = "Classifies the sentiment of a given text."
    model_name: Optional[str] = Field(default="nlptown/bert-base-multilingual-uncased-sentiment")

    # These are not Pydantic fields, they will be set after initialization
    tokenizer: BertTokenizer = None
    model: BertForSequenceClassification = None

    @model_validator(mode="after")
    def load_model(self) -> "SentimentAnalysisTool":
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
        self.model = BertForSequenceClassification.from_pretrained(self.model_name)
        return self

    def _run(self, text: str) -> str:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        outputs = self.model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        sentiment_class = torch.argmax(probs, dim=1).item()

        if sentiment_class <= 1:
            return "Negative"
        elif sentiment_class == 2:
            return "Neutral"
        else:
            return "Positive"

    async def _arun(self, text: str) -> str:
        raise NotImplementedError("Async not implemented yet.")

In [ ]:
# Instantiate the tool
tool = SentimentAnalysisTool()

# Run sentiment analysis
text = "I love this product, it's amazing!"
result = tool._run(text)

print(result)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Positive


## Router Example Code

In [ ]:
# Imports
from langchain.llms import HuggingFaceHub, AzureOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage

# Reuse components from your uploaded file
# llm is already created: (AzureOpenAI or HuggingFaceHub)

# Reuse SentimentAnalysisTool
# Make sure you have this class already imported from your code:
# class SentimentAnalysisTool(BaseTool): ...

sentiment_tool = SentimentAnalysisTool()

# Create a lightweight LLM reasoning function
def llm_reasoning_function(query: str) -> str:
    """Handles general reasoning queries via LLM."""
    return llm.invoke(query)

# Router function
def router(query: str) -> str:
    """
    Decide whether to use SentimentAnalysisTool or LLM based on task.
    If query mentions 'sentiment' or 'classify', route to SentimentAnalysisTool.
    Else, route to LLM.
    """
    keywords = ["sentiment", "classify", "classification"]
    if any(keyword in query.lower() for keyword in keywords):
        return "sentiment_analysis"
    else:
        return "llm_reasoning"

# Set up tools
tools = [
    Tool.from_function(
        name="sentiment_analysis",
        description="Classifies text sentiment using a custom PyTorch model.",
        func=sentiment_tool._run,
    ),
    Tool.from_function(
        name="llm_reasoning",
        description="Handles general reasoning queries with an LLM.",
        func=llm_reasoning_function,
    )
]

# Custom agent prompt
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a hybrid agent that routes tasks either to a sentiment classifier or an LLM."),
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create an agent
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)

# Create an executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Hybrid Execution function
def hybrid_agent_executor(user_query: str):
    selected_tool_name = router(user_query)
    for tool in tools:
        if tool.name == selected_tool_name:
            response = tool.invoke(user_query)
            break
    return response

# Example Usage
query1 = "Can you classify the sentiment of this text: 'I hate rainy days.'?"
query2 = "What's the capital of France?"
query3 = "What's the weather of France today?"

print("Query 1 Result:", hybrid_agent_executor(query1))
print("Query 2 Result:", hybrid_agent_executor(query2))

Query 1 Result: Negative
Query 2 Result: 

The capital of France is Paris.
